<a href="https://colab.research.google.com/github/ewclark47/ADL-Project/blob/main/t5_nmt_en_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install "tensorflow-text>=2.11"
!pip install einops
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import typing
from typing import Any, Tuple

import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tensorflow as tf
import tensorflow_text as tf_text

from transformers import AutoTokenizer, T5ForConditionalGeneration

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
train_raw = tf.data.Dataset.load('/content/gdrive/My Drive/Colab Notebooks/ADL_tokenizer_es_en_translator/small_train_data')
val_raw = tf.data.Dataset.load('/content/gdrive/My Drive/Colab Notebooks/ADL_tokenizer_es_en_translator/small_val_data')

In [6]:
train_raw = train_raw.take(50)

In [7]:
#prefix = "traducir español al inglés: "
prefix = "translate English to Spanish: "

In [8]:
train_sentences_test_target = []
for es,en in train_raw:
  for j in range(len(es.numpy())):
    train_sentences_test_target.append(es.numpy()[j].decode())

train_sentences_test_context = []
for es,en in train_raw:
  for j in range(len(en.numpy())):
    train_sentences_test_context.append(prefix+en.numpy()[j].decode())

In [9]:
df = pd.DataFrame(columns = ['English', 'Spanish'])
df['English'] = train_sentences_test_context
df['Spanish'] = train_sentences_test_target
df.head()

,English,Spanish
0,translate English to Spanish: He asked me if I...,Me preguntó si yo sabía el número de teléfono ...
1,translate English to Spanish: Tom came to my h...,Tom vino a mi casa ayer.
2,translate English to Spanish: She was too prou...,Ella era demasiado orgullosa para pedir su ayuda.
3,translate English to Spanish: I study Korean.,Yo estudio coreano.
4,translate English to Spanish: He got hurt in t...,Él se hirió en el partido de ayer.


In [10]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [11]:
freeze = [model.encoder.block[i].layer[0] for i in range(len(model.encoder.block))] # parts of t5 to freeze
freeze.extend([model.decoder.block[i].layer[0] for i in range(len(model.decoder.block))])
freeze.extend([model.decoder.block[i].layer[1] for i in range(len(model.decoder.block))])

In [12]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [13]:
class DataClass(Dataset):
  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.target_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
        """returns the length of dataframe"""
        return len(self.target_text)

  def __getitem__(self, index):
      """return the input ids, attention masks and target ids"""

      source_text = str(self.source_text[index])
      target_text = str(self.target_text[index])

      # cleaning data so as to ensure data is in string type
      source_text = " ".join(source_text.split())
      target_text = " ".join(target_text.split())

      source = self.tokenizer.batch_encode_plus(
          [source_text],
          max_length=self.source_len,
          pad_to_max_length=True,
          truncation=True,
          padding="max_length",
          return_tensors="pt",
      )
      target = self.tokenizer.batch_encode_plus(
          [target_text],
          max_length=self.target_len,
          pad_to_max_length=True,
          truncation=True,
          padding="max_length",
          return_tensors="pt",
      )

      source_ids = source["input_ids"].squeeze()
      source_mask = source["attention_mask"].squeeze()
      target_ids = target["input_ids"].squeeze()
      target_mask = target["attention_mask"].squeeze()

      return {
          "source_ids": source_ids.to(dtype=torch.long),
          "source_mask": source_mask.to(dtype=torch.long),
          "target_ids": target_ids.to(dtype=torch.long),
          "target_ids_y": target_ids.to(dtype=torch.long),
      }

In [14]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [15]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [16]:
def T5Trainer(
    dataframe, source_text, target_text, model_params
):

    torch.manual_seed(model_params["SEED"]) 
    np.random.seed(model_params["SEED"])
    torch.backends.cudnn.deterministic = True

    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    tokenizer = AutoTokenizer.from_pretrained(model_params["MODEL"])

    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    console.log(f"[Data]: Reading data...\n")

    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    training_set = DataClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = DataClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )
    console.log(f"[Freezing layers]...\n")
    for layer in freeze:
      for param in layer.parameters():
        param.requires_grad = False

    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = '/content/gdrive/My Drive/Colab Notebooks/ADL_tokenizer_es_en_translator/t5_pretrain'
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(path+"/predictions.csv"))


    console.log(f"[Validation Completed.]\n")

    return model

In [17]:
model_params = {
    "MODEL": "t5-small",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 4,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-3,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 128,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 128,  # max length of target text
    "SEED": 47,  # set seed for reproducibility
}

In [18]:
t5_model = T5Trainer(
    dataframe=df,
    source_text='English',
    target_text='Spanish',
    model_params=model_params,
    output_dir="outputs",
)

[21:26:45] [Model]: Loading t5-small...                                           <ipython-input-16-7f31acf18194>:9
                                                                                                                   

[21:26:47] [Data]: Reading data...                                               <ipython-input-16-7f31acf18194>:16
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                        |                      target_text                      |
|---------------------------------------------------------+-------------------------------------------------------|
|translate English to Spanish: He asked me if I knew her  | Me preguntó si yo sabía el número de teléfono de ella.|
|                   telephone number.                     |                                                       |
|   translate English to Spanish: Tom came to my house    |                Tom vino a mi casa ayer.               |
|                       yesterday.                        |                                                       |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (3200, 2)

TRAIN Dataset: (2560, 2)

...

Completed 120

Completed 130

Completed 140

Completed 150

OSError: ignored

### Load the Model

In [19]:
model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/Colab Notebooks/ADL_tokenizer_es_en_translator/t5_pretrain')

In [40]:
test_raw = tf.data.Dataset.load('/content/gdrive/My Drive/Colab Notebooks/ADL_tokenizer_es_en_translator/small_test_data')

In [41]:
#test_raw = test_raw.take(5)
test_raw = test_raw.skip(150)

test_test_target = []
for es,en in test_raw:
  for j in range(len(es.numpy())):
    test_test_target.append(es.numpy()[j].decode())

test_test_context = []
for es,en in test_raw:
  for j in range(len(en.numpy())):
    test_test_context.append(en.numpy()[j].decode())

In [42]:
for i in range(5):
  print('English: ', test_test_context[i])
  print('Spanish: ', test_test_target[i])
  print('\n')

English:  I'm from Kyoto.
Spanish:  Soy de Kioto.


English:  We need less talk and more action.
Spanish:  Necesitamos menos palabras y más acciones.


English:  Tom wants to say something.
Spanish:  Tom quiere decir algo.


English:  Just fill this out.
Spanish:  Rellénalo.


English:  China is much larger than Japan.
Spanish:  China es mucho más grande que Japón.




In [43]:
inputs = tokenizer([prefix+sent for sent in test_test_context], return_tensors='pt', padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask = inputs["attention_mask"],
    do_sample=False
)
test_test_outputs = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [44]:
test_test_outputs[:5]

['Japón de Kyoto.',
 'Tenemos decir más hablar y más a',
 'Tom quiere decir algo.',
 'Seguindo a comprar esto.',
 'China es mucho más grandes que Japón.']

### Test BLEU Scores

In [45]:
from nltk.translate.bleu_score import sentence_bleu

In [46]:
def test_bleu_score_unigram(test_set, generated_output):
  bleu_scores = []
  for sent, gen_sent in zip(test_set, generated_output):
    sent = sent.lower().split(" ")
    #print('Actual Sentence: ', sent)
    gen_sent = gen_sent.lower().split(" ")
    #print('Generated Sentence: ', gen_sent)

    bleu_scores.append(sentence_bleu(sent, gen_sent, weights=(1,0,0,0)))
  
  avg_bleu = sum(bleu_scores)/len(bleu_scores)

  return avg_bleu

In [47]:
result_bleu = test_bleu_score_unigram(test_test_target, test_test_outputs)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [48]:
print(f"BLEU Score for T5 is: {result_bleu}")

BLEU Score for T5 is: 0.04441412366618372


# Reference:
https://shivanandroy.com/fine-tune-t5-transformer-with-pytorch/